### Train Model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os

In [2]:
from collections import Counter

In [3]:
from sklearn.compose import ColumnTransformer

In [4]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import VarianceThreshold

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [6]:
from sklearn.utils import compute_sample_weight
from sklearn.utils import class_weight

In [7]:
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer, accuracy_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
from sklearn.metrics import classification_report

In [8]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline, make_pipeline

In [9]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

In [10]:
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
from sklearn.metrics import classification_report

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [12]:
# Models
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [13]:
# Set display and format settings

pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)
pd.options.display.float_format = '{:.0f}'.format

In [14]:
DATA_ROOT_DIR='/mnt/data/projects/MD7'
PROJ_ROOT_DIR='/home/priyesh/projects/MD7'

### Load Data

In [33]:
filepath=os.path.join(PROJ_ROOT_DIR,'pickle','yahoo_complete_flat.pkl')
df_complete_flat = pd.read_pickle(filepath)

In [34]:
filepath=os.path.join(PROJ_ROOT_DIR,'pickle','yahoo_complete.pkl')
df_complete = pd.read_pickle(filepath)

In [35]:
# Take a copy so that we can always refer back to original 

df = df_complete_flat.copy()

In [36]:
df.head()

,company,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_bal_totalDebt,0_bal_totalAssets,0_bal_totalLiabilities,0_bal_workingCapital,0_bal_quickRatio,0_bal_currentRatio,0_bal_debtCapitalRatio,0_bal_debtAssetRatio,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfPPE,0_cas_cashFlowInvestingActivities,0_cas_cashDividendsPaid,0_cas_repurchaseOfCapitalStock,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_bal_totalDebt,1_bal_totalAssets,1_bal_totalLiabilities,1_bal_workingCapital,1_bal_quickRatio,1_bal_currentRatio,1_bal_debtCapitalRatio,1_bal_debtAssetRatio,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfPPE,1_cas_cashFlowInvestingActivities,1_cas_cashDividendsPaid,1_cas_repurchaseOfCapitalStock,1_cas_cashFlowFinancingActivities,1_cas_freeCashFlow,2_inc_totalRevenue,2_inc_costOfRevenue,2_inc_grossProfit,2_inc_netIncome,2_inc_operatingIncome,2_inc_netIncomeContinuousOperations,2_inc_netInterestIncome,2_inc_interestIncome,2_inc_otherIncomeExpense,2_inc_operatingExpense,2_inc_totalExpenses,2_inc_taxProvision,2_inc_interestExpense,2_inc_SGA,2_inc_researchAndDevelopment,2_inc_ebit,2_inc_dilutedEPS,2_inc_basicEPS,2_bal_cashEquivalent,2_bal_receivables,2_bal_finishedGoods,2_bal_workInProcess,2_bal_rawMaterials,2_bal_otherCurrentAssets,2_bal_inventory,2_bal_currentAssets,2_bal_netPPE,2_bal_otherNonCurrentAssets,2_bal_financialAssets,2_bal_goodwill,2_bal_goodwillAndOtherIntangibleAssets,2_bal_otherIntangibleAssets,2_bal_nonCurrentAccountsReceivable,2_bal_totalNonCurrentAssets,2_bal_currentDebt,2_bal_payablesAndAccruedExpenses,2_bal_otherCurrentLiabilities,2_bal_currentLiabilities,2_bal_longTermDebt,2_bal_otherNonCurrentLiabilities,2_bal_nonCurrentDeferredLiabilities,2_bal_nonCurrentD

In [37]:
# Rename final score to label

df.rename(columns={'FinalScore':'label'},inplace=True)

In [38]:
# Drop column company

df.drop('company',axis=1,inplace=True)

In [39]:
df.head()

,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_bal_totalDebt,0_bal_totalAssets,0_bal_totalLiabilities,0_bal_workingCapital,0_bal_quickRatio,0_bal_currentRatio,0_bal_debtCapitalRatio,0_bal_debtAssetRatio,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfPPE,0_cas_cashFlowInvestingActivities,0_cas_cashDividendsPaid,0_cas_repurchaseOfCapitalStock,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_bal_totalDebt,1_bal_totalAssets,1_bal_totalLiabilities,1_bal_workingCapital,1_bal_quickRatio,1_bal_currentRatio,1_bal_debtCapitalRatio,1_bal_debtAssetRatio,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfPPE,1_cas_cashFlowInvestingActivities,1_cas_cashDividendsPaid,1_cas_repurchaseOfCapitalStock,1_cas_cashFlowFinancingActivities,1_cas_freeCashFlow,2_inc_totalRevenue,2_inc_costOfRevenue,2_inc_grossProfit,2_inc_netIncome,2_inc_operatingIncome,2_inc_netIncomeContinuousOperations,2_inc_netInterestIncome,2_inc_interestIncome,2_inc_otherIncomeExpense,2_inc_operatingExpense,2_inc_totalExpenses,2_inc_taxProvision,2_inc_interestExpense,2_inc_SGA,2_inc_researchAndDevelopment,2_inc_ebit,2_inc_dilutedEPS,2_inc_basicEPS,2_bal_cashEquivalent,2_bal_receivables,2_bal_finishedGoods,2_bal_workInProcess,2_bal_rawMaterials,2_bal_otherCurrentAssets,2_bal_inventory,2_bal_currentAssets,2_bal_netPPE,2_bal_otherNonCurrentAssets,2_bal_financialAssets,2_bal_goodwill,2_bal_goodwillAndOtherIntangibleAssets,2_bal_otherIntangibleAssets,2_bal_nonCurrentAccountsReceivable,2_bal_totalNonCurrentAssets,2_bal_currentDebt,2_bal_payablesAndAccruedExpenses,2_bal_otherCurrentLiabilities,2_bal_currentLiabilities,2_bal_longTermDebt,2_bal_otherNonCurrentLiabilities,2_bal_nonCurrentDeferredLiabilities,2_bal_nonCurrentDeferredT

In [40]:
df['label'].value_counts()

label
3    315
2    273
1     83
4     29
0     28
Name: count, dtype: int64

### Split Data into Independent and Dependent Columns

In [41]:
X = df.drop('label',axis=1)
y = df['label']

In [42]:
print(X.shape,y.shape)

(728, 260) (728,)


## Train Test Split

In [43]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=66,shuffle=True,stratify=y)

In [44]:
X_train.head()

,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_bal_totalDebt,0_bal_totalAssets,0_bal_totalLiabilities,0_bal_workingCapital,0_bal_quickRatio,0_bal_currentRatio,0_bal_debtCapitalRatio,0_bal_debtAssetRatio,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfPPE,0_cas_cashFlowInvestingActivities,0_cas_cashDividendsPaid,0_cas_repurchaseOfCapitalStock,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_bal_totalDebt,1_bal_totalAssets,1_bal_totalLiabilities,1_bal_workingCapital,1_bal_quickRatio,1_bal_currentRatio,1_bal_debtCapitalRatio,1_bal_debtAssetRatio,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfPPE,1_cas_cashFlowInvestingActivities,1_cas_cashDividendsPaid,1_cas_repurchaseOfCapitalStock,1_cas_cashFlowFinancingActivities,1_cas_freeCashFlow,2_inc_totalRevenue,2_inc_costOfRevenue,2_inc_grossProfit,2_inc_netIncome,2_inc_operatingIncome,2_inc_netIncomeContinuousOperations,2_inc_netInterestIncome,2_inc_interestIncome,2_inc_otherIncomeExpense,2_inc_operatingExpense,2_inc_totalExpenses,2_inc_taxProvision,2_inc_interestExpense,2_inc_SGA,2_inc_researchAndDevelopment,2_inc_ebit,2_inc_dilutedEPS,2_inc_basicEPS,2_bal_cashEquivalent,2_bal_receivables,2_bal_finishedGoods,2_bal_workInProcess,2_bal_rawMaterials,2_bal_otherCurrentAssets,2_bal_inventory,2_bal_currentAssets,2_bal_netPPE,2_bal_otherNonCurrentAssets,2_bal_financialAssets,2_bal_goodwill,2_bal_goodwillAndOtherIntangibleAssets,2_bal_otherIntangibleAssets,2_bal_nonCurrentAccountsReceivable,2_bal_totalNonCurrentAssets,2_bal_currentDebt,2_bal_payablesAndAccruedExpenses,2_bal_otherCurrentLiabilities,2_bal_currentLiabilities,2_bal_longTermDebt,2_bal_otherNonCurrentLiabilities,2_bal_nonCurrentDeferredLiabilities,2_bal_nonCurrentDeferredT

In [45]:
X_test.head()

,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_bal_totalDebt,0_bal_totalAssets,0_bal_totalLiabilities,0_bal_workingCapital,0_bal_quickRatio,0_bal_currentRatio,0_bal_debtCapitalRatio,0_bal_debtAssetRatio,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfPPE,0_cas_cashFlowInvestingActivities,0_cas_cashDividendsPaid,0_cas_repurchaseOfCapitalStock,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_bal_totalDebt,1_bal_totalAssets,1_bal_totalLiabilities,1_bal_workingCapital,1_bal_quickRatio,1_bal_currentRatio,1_bal_debtCapitalRatio,1_bal_debtAssetRatio,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfPPE,1_cas_cashFlowInvestingActivities,1_cas_cashDividendsPaid,1_cas_repurchaseOfCapitalStock,1_cas_cashFlowFinancingActivities,1_cas_freeCashFlow,2_inc_totalRevenue,2_inc_costOfRevenue,2_inc_grossProfit,2_inc_netIncome,2_inc_operatingIncome,2_inc_netIncomeContinuousOperations,2_inc_netInterestIncome,2_inc_interestIncome,2_inc_otherIncomeExpense,2_inc_operatingExpense,2_inc_totalExpenses,2_inc_taxProvision,2_inc_interestExpense,2_inc_SGA,2_inc_researchAndDevelopment,2_inc_ebit,2_inc_dilutedEPS,2_inc_basicEPS,2_bal_cashEquivalent,2_bal_receivables,2_bal_finishedGoods,2_bal_workInProcess,2_bal_rawMaterials,2_bal_otherCurrentAssets,2_bal_inventory,2_bal_currentAssets,2_bal_netPPE,2_bal_otherNonCurrentAssets,2_bal_financialAssets,2_bal_goodwill,2_bal_goodwillAndOtherIntangibleAssets,2_bal_otherIntangibleAssets,2_bal_nonCurrentAccountsReceivable,2_bal_totalNonCurrentAssets,2_bal_currentDebt,2_bal_payablesAndAccruedExpenses,2_bal_otherCurrentLiabilities,2_bal_currentLiabilities,2_bal_longTermDebt,2_bal_otherNonCurrentLiabilities,2_bal_nonCurrentDeferredLiabilities,2_bal_nonCurrentDeferredT

## Prepare Data

### Class Imbalance

In [28]:
df = pd.concat([X_train,y_train],axis=1)
df.head()

,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_bal_totalDebt,0_bal_totalAssets,0_bal_totalLiabilities,0_bal_workingCapital,0_bal_quickRatio,0_bal_currentRatio,0_bal_debtCapitalRatio,0_bal_debtAssetRatio,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfPPE,0_cas_cashFlowInvestingActivities,0_cas_cashDividendsPaid,0_cas_repurchaseOfCapitalStock,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_bal_totalDebt,1_bal_totalAssets,1_bal_totalLiabilities,1_bal_workingCapital,1_bal_quickRatio,1_bal_currentRatio,1_bal_debtCapitalRatio,1_bal_debtAssetRatio,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfPPE,1_cas_cashFlowInvestingActivities,1_cas_cashDividendsPaid,1_cas_repurchaseOfCapitalStock,1_cas_cashFlowFinancingActivities,1_cas_freeCashFlow,2_inc_totalRevenue,2_inc_costOfRevenue,2_inc_grossProfit,2_inc_netIncome,2_inc_operatingIncome,2_inc_netIncomeContinuousOperations,2_inc_netInterestIncome,2_inc_interestIncome,2_inc_otherIncomeExpense,2_inc_operatingExpense,2_inc_totalExpenses,2_inc_taxProvision,2_inc_interestExpense,2_inc_SGA,2_inc_researchAndDevelopment,2_inc_ebit,2_inc_dilutedEPS,2_inc_basicEPS,2_bal_cashEquivalent,2_bal_receivables,2_bal_finishedGoods,2_bal_workInProcess,2_bal_rawMaterials,2_bal_otherCurrentAssets,2_bal_inventory,2_bal_currentAssets,2_bal_netPPE,2_bal_otherNonCurrentAssets,2_bal_financialAssets,2_bal_goodwill,2_bal_goodwillAndOtherIntangibleAssets,2_bal_otherIntangibleAssets,2_bal_nonCurrentAccountsReceivable,2_bal_totalNonCurrentAssets,2_bal_currentDebt,2_bal_payablesAndAccruedExpenses,2_bal_otherCurrentLiabilities,2_bal_currentLiabilities,2_bal_longTermDebt,2_bal_otherNonCurrentLiabilities,2_bal_nonCurrentDeferredLiabilities,2_bal_nonCurrentDeferredT

In [29]:
df = pd.concat([X_train,y_train],axis=1)

df_class_0 = df[df['label'] == 0]
df_class_1 = df[df['label'] == 1]
df_class_2 = df[df['label'] == 2]
df_class_3 = df[df['label'] == 3]
df_class_4 = df[df['label'] == 4]

df_class_0_over = df_class_0.sample(252, replace=True)
df_class_1_over = df_class_1.sample(252, replace=True)
df_class_2_over = df_class_2.sample(252, replace=True)
df_class_4_over = df_class_4.sample(252, replace=True)

df_temp = pd.concat([df_class_0_over,df_class_1_over,df_class_2_over,df_class_3,df_class_4_over],axis=0)

X_train = df_temp.drop('label',axis=1)
y_train = df_temp['label']

In [30]:
y_train.value_counts()

label
0    252
1    252
2    252
3    252
4    252
Name: count, dtype: int64

### One Hot Encoding

In [ ]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ct = ColumnTransformer(
    [("ohe", ohe, [0]),
    ],remainder='passthrough')    

In [ ]:
#X_train = ct.fit_transform(X_train)
#X_test = ct.transform(X_test)

### Robust Scaler

In [54]:
robustScaler=RobustScaler()

In [55]:
l = X_train.shape[1]
ct1 = ColumnTransformer(
          [('robustScaler',robustScaler, slice(0,l))
          ],remainder='passthrough')   

In [56]:
# Apply StandardScaler to All Columns

X_train_robustScale = ct1.fit_transform(X_train)
X_test_robustScale = ct1.transform(X_test)

In [57]:
X_test_robustScale

array([[ 8.76428650e-01,  3.81611179e-01,  2.19186663e+00, ...,
         4.55612207e+00,  4.99056572e+00,  4.70436810e+00],
       [ 1.50441935e+00,  1.08057087e+00,  2.77031890e+00, ...,
         4.92978719e+00,  4.88236848e+00,  4.33376450e+00],
       [-9.36802448e-02, -1.01685355e-01,  9.86715726e-02, ...,
         2.15755703e-01, -1.30351366e-01, -4.12727319e-03],
       ...,
       [ 7.15966634e-01,  1.19226410e+00,  5.44420439e-02, ...,
         6.64286823e-01, -1.88278541e+00, -8.55196535e-01],
       [-4.10607503e-01, -2.98747711e-01, -5.06371779e-01, ...,
        -2.50993254e-01, -3.14082522e-01, -4.96847104e-01],
       [-4.40037454e-01, -3.74564983e-01, -4.46871798e-01, ...,
        -2.50993254e-01, -2.96408998e-01, -3.27986316e-01]])

### Outliers - IQR

In [ ]:
x = list(X_train.columns)
x.sort
x

In [59]:
def cap_ext_outliers(train_X,test_X):

  df_train = train_X.copy()
  df_test = test_X.copy()
    
  cols = df_train.columns
  cols = list(cols)

  for feature in cols:
    IQR = X_train[feature].quantile(0.75)- X_train[feature].quantile(0.25)
    ext_lower_bound = df_train[feature].quantile(0.75) - (IQR*3)
    ext_upper_bound = df_train[feature].quantile(0.75) + (IQR*3) 
      
    df_train[feature] = np.where(df_train[feature] < ext_lower_bound, ext_lower_bound, df_train[feature])
    df_train[feature] = np.where(df_train[feature] > ext_upper_bound, ext_upper_bound, df_train[feature])  

    df_test[feature] = np.where(df_test[feature] < ext_lower_bound, ext_lower_bound, df_test[feature])
    df_test[feature] = np.where(df_test[feature] > ext_upper_bound, ext_upper_bound, df_test[feature])  

  return df_train,df_test


In [60]:
def cap_outliers(train_X,test_X):

  df_train = train_X.copy()
  df_test = test_X.copy()
    
  cols = df_train.columns
  cols = list(cols)

  for feature in cols:
    IQR = X_train[feature].quantile(0.75)- X_train[feature].quantile(0.25)
    ext_lower_bound = df_train[feature].quantile(0.75) - (IQR*1.5)
    ext_upper_bound = df_train[feature].quantile(0.75) + (IQR*1.5) 
      
    df_train[feature] = np.where(df_train[feature] < ext_lower_bound, ext_lower_bound, df_train[feature])
    df_train[feature] = np.where(df_train[feature] > ext_upper_bound, ext_upper_bound, df_train[feature])  

    df_test[feature] = np.where(df_test[feature] < ext_lower_bound, ext_lower_bound, df_test[feature])
    df_test[feature] = np.where(df_test[feature] > ext_upper_bound, ext_upper_bound, df_test[feature])  

  return df_train,df_test

In [61]:
X_train_cap, X_test_cap = cap_outliers(X_train,X_test)

In [ ]:
#X_train_ext_cap, X_test_ext_cap = cap_ext_outliers(X_train,X_test)

In [122]:
X_train_cap.shape

(1260, 260)

,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_bal_totalDebt,0_bal_totalAssets,0_bal_totalLiabilities,0_bal_workingCapital,0_bal_quickRatio,0_bal_currentRatio,0_bal_debtCapitalRatio,0_bal_debtAssetRatio,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfPPE,0_cas_cashFlowInvestingActivities,0_cas_cashDividendsPaid,0_cas_repurchaseOfCapitalStock,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_bal_totalDebt,1_bal_totalAssets,1_bal_totalLiabilities,1_bal_workingCapital,1_bal_quickRatio,1_bal_currentRatio,1_bal_debtCapitalRatio,1_bal_debtAssetRatio,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfPPE,1_cas_cashFlowInvestingActivities,1_cas_cashDividendsPaid,1_cas_repurchaseOfCapitalStock,1_cas_cashFlowFinancingActivities,1_cas_freeCashFlow,2_inc_totalRevenue,2_inc_costOfRevenue,2_inc_grossProfit,2_inc_netIncome,2_inc_operatingIncome,2_inc_netIncomeContinuousOperations,2_inc_netInterestIncome,2_inc_interestIncome,2_inc_otherIncomeExpense,2_inc_operatingExpense,2_inc_totalExpenses,2_inc_taxProvision,2_inc_interestExpense,2_inc_SGA,2_inc_researchAndDevelopment,2_inc_ebit,2_inc_dilutedEPS,2_inc_basicEPS,2_bal_cashEquivalent,2_bal_receivables,2_bal_finishedGoods,2_bal_workInProcess,2_bal_rawMaterials,2_bal_otherCurrentAssets,2_bal_inventory,2_bal_currentAssets,2_bal_netPPE,2_bal_otherNonCurrentAssets,2_bal_financialAssets,2_bal_goodwill,2_bal_goodwillAndOtherIntangibleAssets,2_bal_otherIntangibleAssets,2_bal_nonCurrentAccountsReceivable,2_bal_totalNonCurrentAssets,2_bal_currentDebt,2_bal_payablesAndAccruedExpenses,2_bal_otherCurrentLiabilities,2_bal_currentLiabilities,2_bal_longTermDebt,2_bal_otherNonCurrentLiabilities,2_bal_nonCurrentDeferredLiabilities,2_bal_nonCurrentDeferredT

### PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=75)

In [ ]:
X_train_pca = pca.fit_transform(X_train_scale)
X_test_pca = pca.transform(X_test_scale)

In [ ]:
X_train_pca.shape

### Smote

In [ ]:
l = X_train_fsel_mic.shape[1]
stdScale=stdScale=StandardScaler()
trfScaleStd = ColumnTransformer(
                  [('stdScale',stdScale, slice(0,l))
                  ],remainder='passthrough')   

In [ ]:
trfSmote = SMOTE()

In [ ]:
pipe1 = Pipeline([
     ('trf1',trfScaleStd),
])

In [ ]:
pipe2 = Pipeline([
     ('trf2',trfSmote)
])

In [ ]:
X_train_temp = pipe1.fit_transform(X_train_fsel_mic)

In [ ]:
X_train_smote, y_train_smote = pipe2.fit_resample(X_train_temp,y_train)

In [ ]:
X_train_smote.shape

In [ ]:
counter = Counter(y_train_smote)
for k, v in counter.items():
  dist = v / len(y_train_smote) * 100
  print(f"Class={k}, n={v} ({dist}%)")

In [ ]:
y_train_smote

In [ ]:
### Feature Selection - Variance

In [124]:
X_train_sel1.shape

(1260, 244)

In [126]:
X_train_sel1

array([[ 2.45862054e+08,  1.31664713e+08,  1.14197341e+08, ...,
         0.00000000e+00, -3.36538000e+08, -2.73298000e+08],
       [ 5.47100000e+09,  1.97780000e+09,  3.49320000e+09, ...,
         0.00000000e+00,  9.13000000e+07, -8.48300000e+08],
       [ 2.45862054e+08,  1.31664713e+08,  1.14197341e+08, ...,
         0.00000000e+00, -3.36538000e+08, -2.73298000e+08],
       ...,
       [ 7.14006000e+08,  2.40515000e+08,  4.73491000e+08, ...,
         1.60950000e+08,  2.42853000e+08,  2.80364000e+08],
       [ 5.51790000e+09,  3.54430000e+09,  1.97360000e+09, ...,
         2.59800000e+08,  3.70000000e+08,  1.59710000e+09],
       [ 4.59000000e+09,  3.41600000e+09,  1.17400000e+09, ...,
         9.22500000e+08,  2.07162050e+09,  2.55800850e+09]])

### Feature Selection - KBest

In [ ]:
trfFSel = SelectKBest(score_func=chi2,k=200)
trfScaleMinMax = MinMaxScaler()

In [ ]:
pipe3 = Pipeline([
     ('trf1',trfScaleMinMax),
     ('trf2',trfFSel),   
])

In [ ]:
X_train_selKBest = pipe3.fit_transform(X_train,y_train)
X_test_selKBest  = pipe3.fit(X_test,y_test)

### Feature Selection - Correlation

In [62]:
X_train_cap.head()

,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_bal_totalDebt,0_bal_totalAssets,0_bal_totalLiabilities,0_bal_workingCapital,0_bal_quickRatio,0_bal_currentRatio,0_bal_debtCapitalRatio,0_bal_debtAssetRatio,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfPPE,0_cas_cashFlowInvestingActivities,0_cas_cashDividendsPaid,0_cas_repurchaseOfCapitalStock,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_bal_totalDebt,1_bal_totalAssets,1_bal_totalLiabilities,1_bal_workingCapital,1_bal_quickRatio,1_bal_currentRatio,1_bal_debtCapitalRatio,1_bal_debtAssetRatio,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfPPE,1_cas_cashFlowInvestingActivities,1_cas_cashDividendsPaid,1_cas_repurchaseOfCapitalStock,1_cas_cashFlowFinancingActivities,1_cas_freeCashFlow,2_inc_totalRevenue,2_inc_costOfRevenue,2_inc_grossProfit,2_inc_netIncome,2_inc_operatingIncome,2_inc_netIncomeContinuousOperations,2_inc_netInterestIncome,2_inc_interestIncome,2_inc_otherIncomeExpense,2_inc_operatingExpense,2_inc_totalExpenses,2_inc_taxProvision,2_inc_interestExpense,2_inc_SGA,2_inc_researchAndDevelopment,2_inc_ebit,2_inc_dilutedEPS,2_inc_basicEPS,2_bal_cashEquivalent,2_bal_receivables,2_bal_finishedGoods,2_bal_workInProcess,2_bal_rawMaterials,2_bal_otherCurrentAssets,2_bal_inventory,2_bal_currentAssets,2_bal_netPPE,2_bal_otherNonCurrentAssets,2_bal_financialAssets,2_bal_goodwill,2_bal_goodwillAndOtherIntangibleAssets,2_bal_otherIntangibleAssets,2_bal_nonCurrentAccountsReceivable,2_bal_totalNonCurrentAssets,2_bal_currentDebt,2_bal_payablesAndAccruedExpenses,2_bal_otherCurrentLiabilities,2_bal_currentLiabilities,2_bal_longTermDebt,2_bal_otherNonCurrentLiabilities,2_bal_nonCurrentDeferredLiabilities,2_bal_nonCurrentDeferredT

In [63]:
pd.options.display.float_format = '{:.2f}'.format
df_temp = X_train_cap.corr()
df_temp[(df_temp > 0.95) & (df_temp != 1)].stack()

0_inc_totalRevenue                        0_inc_costOfRevenue                        0.95
                                          0_inc_totalExpenses                        0.99
                                          1_inc_totalRevenue                         0.97
                                          1_inc_totalExpenses                        0.98
                                          2_inc_totalRevenue                         0.96
                                          2_inc_totalExpenses                        0.96
                                          3_inc_totalRevenue                         0.95
                                          3_inc_totalExpenses                        0.95
0_inc_costOfRevenue                       0_inc_totalRevenue                         0.95
                                          0_inc_totalExpenses                        0.96
                                          1_inc_costOfRevenue                        0.98
          

In [64]:
# Select highly correlated features

def correlation(dataset, threshold):
  col_corr = set()
  corr_matrix = dataset.corr()
  for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if corr_matrix.iloc[i,j] > threshold:
          colname = corr_matrix.columns[i]
          col_corr.add(colname)
  return col_corr

In [65]:
corr_features = correlation(X_train_cap,0.90)
len(set(corr_features))

152

In [66]:
X_train_sel2 = X_train.drop(corr_features,axis=1)
X_test_sel2 = X_test.drop(corr_features,axis=1)

In [67]:
X_train_sel2.shape

(582, 108)

In [68]:
X_test_sel2.shape

(146, 108)

### Feature Selection - Mutual Info Classification

In [69]:
from sklearn.feature_selection import mutual_info_classif

mutual_info = mutual_info_classif(X_train_sel2, y_train)
#mutual_info

In [70]:
pd.options.display.float_format = '{:.2f}'.format
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train_sel2.columns
mutual_info.sort_values(ascending=False)
df_temp = pd.DataFrame(mutual_info)
df_temp = df_temp.reset_index()
df_temp.columns = ['feature','value']
df_temp.sort_values(by='value',ascending=False).reset_index(drop=True)


,feature,value
0,2_inc_netIncome,0.23
1,2_inc_dilutedEPS,0.23
2,1_cas_cashFlowOperatingActivities,0.23
3,3_cas_freeCashFlow,0.21
4,3_inc_taxProvision,0.20
5,2_cas_freeCashFlow,0.19
6,3_inc_dilutedEPS,0.19
7,3_inc_netIncome,0.19
8,2_inc_taxProvision,0.16
9,1_inc_netIncome,0.15


In [71]:
from sklearn.feature_selection import SelectKBest
# Select Top 100 features

sel_top_cols = SelectKBest(mutual_info_classif, k=100)
sel_top_cols.fit(X_train_sel2,y_train)
cols = X_train_sel2.columns[sel_top_cols.get_support()]

In [72]:
cols

Index(['0_inc_totalRevenue', '0_inc_grossProfit', '0_inc_netIncome',
       '0_inc_operatingIncome', '0_inc_netInterestIncome',
       '0_inc_otherIncomeExpense', '0_inc_taxProvision',
       '0_inc_interestExpense', '0_inc_researchAndDevelopment',
       '0_inc_dilutedEPS', '0_bal_cashEquivalent', '0_bal_receivables',
       '0_bal_finishedGoods', '0_bal_workInProcess', '0_bal_rawMaterials',
       '0_bal_otherCurrentAssets', '0_bal_inventory', '0_bal_netPPE',
       '0_bal_otherNonCurrentAssets', '0_bal_goodwill',
       '0_bal_otherIntangibleAssets', '0_bal_nonCurrentAccountsReceivable',
       '0_bal_totalNonCurrentAssets', '0_bal_otherCurrentLiabilities',
       '0_bal_otherNonCurrentLiabilities',
       '0_bal_nonCurrentDeferredLiabilities',
       '0_bal_nonCurrentDeferredTaxesLiabilities', '0_bal_longTermProvisions',
       '0_bal_retainedEarnings', '0_bal_stockholdersEquity',
       '0_bal_workingCapital', '0_bal_quickRatio', '0_bal_currentRatio',
       '0_bal_debtCapitalRati

In [73]:
X_train_sel2.shape

(582, 108)

In [74]:
X_train_fsel_mic = X_train_sel2[cols]
X_test_fsel_mic = X_test_sel2[cols]

In [75]:
X_train_fsel_mic.shape

(582, 100)

In [76]:
X_test_fsel_mic.shape

(146, 100)

In [77]:
X_train_fsel_mic.head()

,0_inc_totalRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netInterestIncome,0_inc_otherIncomeExpense,0_inc_taxProvision,0_inc_interestExpense,0_inc_researchAndDevelopment,0_inc_dilutedEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_goodwill,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_otherCurrentLiabilities,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_bal_workingCapital,0_bal_quickRatio,0_bal_currentRatio,0_bal_debtCapitalRatio,0_bal_debtAssetRatio,0_cas_stockBasedCompensation,0_cas_saleOfBusiness,0_cas_purchaseOfPPE,0_cas_cashDividendsPaid,0_cas_repurchaseOfCapitalStock,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_netIncome,1_inc_operatingIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_taxProvision,1_inc_dilutedEPS,1_bal_cashEquivalent,1_bal_nonCurrentAccountsReceivable,1_bal_currentDebt,1_bal_longTermProvisions,1_bal_workingCapital,1_bal_quickRatio,1_bal_currentRatio,1_bal_debtCapitalRatio,1_bal_debtAssetRatio,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_repurchaseOfCapitalStock,1_cas_cashFlowFinancingActivities,1_cas_freeCashFlow,2_inc_netIncome,2_inc_otherIncomeExpense,2_inc_taxProvision,2_inc_dilutedEPS,2_bal_financialAssets,2_bal_nonCurrentAccountsReceivable,2_bal_currentDebt,2_bal_otherCurrentLiabilities,2_bal_longTermProvisions,2_bal_workingCapital,2_bal_quickRatio,2_bal_currentRatio,2_bal_debtCapitalRatio,2_bal_debtAssetRatio,2_cas_saleOfBusiness,2_cas_cashFlowInvestingActivities,2_cas_repurchaseOfCapitalStock,2_cas_cashFlowFinancingActivities,2_cas_freeCashFlow,3_inc_netIncome,3_inc_netInterestIncome,3_inc_interestIncome,3_inc_otherIncomeExpense,3_inc_taxProvision,3_inc_dilutedEPS,3_bal_otherCurrentAssets,3_bal_financialAssets,3_bal_nonCurrentAccountsReceivable,3_bal_currentDebt,3_bal_longTermProvisions,3_bal_workingCapital,3_bal_quickRatio,3_bal_currentRatio,3_cas_saleOfBusiness,3_cas_cashFlowInvestingActivities,3_cas_repurchaseOfCapitalStock,3_cas_cashFlowFinancingActivities,3_cas_freeCashFlow
228,5333700000,2515400000,790900000,1056000000,-13500000,1200000,252800000,13900000,0,1,174900000,741800000,0,0,0,157400000,1366400000,1266400000,76300000,0,0,0,1342700000,0,0,99400000,99400000,0,2633900000,2665600000,1912500000,2,4,18,15,5700000,0,246400000,498600000,0,595100000,596300000,859100000,1140400000,600000,1400000,273600000,1,245700000,0,133600000,0,1886900000,1,4,19,16,5700000,1101800000,52000000,754400000,933700000,925000000,0,282800000,1,0,0,150800000,0,0,2174400000,1,4,17,14,0,148500000,0,627100000,613500000,1086900000,-13600000,700000,0,353100000,1,165400000,0,0,293700000,0,2335000000,1,3,0,163000000,237800000,774900000,767200000
399,4200819000,2518585000,1107835000,1402939000,13023000,13023000,308127000,0,0,1,1331020000,540330000,225846000,0,134885000,0,360731000,298640000,53973000,1331643000,1052105000,0,2834043000,0,30505000,287469000,0,0,5022480000,4171281000,1655212000,2,3,0,0,63356000,0,101661000,0,707300000,628506000,1003364000,1409594000,1633153000,0,-6996000,216563000,1,2061767000,0,0,0,2390967000,3,4,0,0,70289000,1364163000,595918000,526068000,1296891000,1377475000,3952000,423944000,1,0,0,0,0,0,3717034000,4,4,0,0,0,992022000,13830000,-34821000,1098288000,1191624000,0,0,-12757000,380340000,1,109823000,0,0,0,0,3762919000,3,4,0,161367000,771028000,706938000,675546000
249,2988100000,1961300000,137000000,211300000,-92100000,7200000,-12000000,92100000,492600000,0,1086400000,30200000,0,0,0,48900000,0,455200000,17200000,2976500000,1097700000,0,4725900000,93800000,17700000,755300000,100900000,0,-153500000,772000000,-425800000,0,0,77,41,147000000,0,87600000,0,458600000,456900000,631100000,-495100000,-358900000

### Feature Selection - Variance

In [156]:
#var_thres=VarianceThreshold(threshold=0)
#X_train_fsel_var = var_thres.fit_transform(X_train_fsel_mic)
#X_test_fsel_var = var_thres.transform(X_test_fsel_mic)

In [157]:
#X_train_fsel_var.shape

(1260, 100)

### Standard Scaler

In [159]:
stdScaler=StandardScaler()
l = X_train.shape[1]
ct1 = ColumnTransformer(
          [('stdScaler',stdScaler, slice(0,l))
          ],remainder='passthrough')   

In [160]:
# Apply StandardScaler to All Columns

X_train_fsel_scale = ct1.fit_transform(X_train_fsel_mic)
X_test_fsel_scale = ct1.transform(X_test_fsel_mic)

## Model Selection

In [46]:
def train_eval_model(model,train_X,train_y,test_X,test_y,use_sample_weights=False):

  if use_sample_weights:
    class_weights = class_weight.compute_sample_weight(
            class_weight='balanced',
            y=train_y)
    model.fit(train_X, train_y,sample_weight = class_weights)
  else:
    model.fit(train_X, train_y)  
      
  y_pred = model.predict(test_X)
  y_train_predict = model.predict(train_X)

  train_accuracy = round(accuracy_score(train_y, y_train_predict),3)
  test_accuracy = round(accuracy_score(test_y, y_pred),3)

  train_f1 = round(f1_score(train_y, y_train_predict,zero_division=np.nan, average='weighted'),3)
  test_f1 = round(f1_score(test_y, y_pred,zero_division=np.nan, average='weighted'),3)
    
  print('Train accuracy',train_accuracy)
  print('Test accuracy',test_accuracy) 
  print
    
  print('Train F1 Score',train_f1)
  print('Test F1 Score',test_f1) 
  print("")

  cross_val_accuracy = round(np.mean(cross_val_score(model,train_X,train_y,scoring='accuracy',cv=5)),3)
 
  cross_val_f1 = round(np.mean(cross_val_score(model,train_X,train_y,
                                         scoring=make_scorer(f1_score, average='weighted'),
                                         cv=5)),3)
  
  print('Cross val accuracy:',cross_val_accuracy)
  print('Cross val f1:',cross_val_f1)
  print("")
  print(pd.crosstab(test_y,y_pred))
  print("")
  print(classification_report(test_y, y_pred))


### Basesline Evaluation

Default parameters, no modifications to dataset.

#### XGBoost

In [47]:
model = xgb.XGBClassifier(
                       random_state=0)      

train_eval_model(model, X_train, y_train, X_test, y_test)


Train accuracy 1.0
Test accuracy 0.685
Train F1 Score 1.0
Test F1 Score 0.696

Cross val accuracy: 0.649
Cross val f1: 0.632

col_0  0   1   2   3  4
label                  
0      5   0   0   0  0
1      1  10   6   0  0
2      0   4  32  19  0
3      0   1   8  53  1
4      0   0   1   5  0

              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       0.67      0.59      0.62        17
           2       0.68      0.58      0.63        55
           3       0.69      0.84      0.76        63
           4       0.00      0.00      0.00         6

    accuracy                           0.68       146
   macro avg       0.57      0.60      0.58       146
weighted avg       0.66      0.68      0.67       146



In [1]:
model = xgb.XGBClassifier(
                       random_state=50)      

train_eval_model(model, X_train, y_train, X_test, y_test,use_sample_weights=True)

NameError: name 'xgb' is not defined

### Adaboost

In [49]:
model = AdaBoostClassifier(n_estimators=200,algorithm='SAMME')
train_eval_model(model, X_train, y_train, X_test, y_test)

Train accuracy 0.603
Test accuracy 0.527
Train F1 Score 0.579
Test F1 Score 0.55

Cross val accuracy: 0.551
Cross val f1: 0.536

col_0  0   1   2   3
label               
0      0   5   0   0
1      1  11   5   0
2      0   5  35  15
3      0   1  31  31
4      0   0   2   4

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.50      0.65      0.56        17
           2       0.48      0.64      0.55        55
           3       0.62      0.49      0.55        63
           4       0.00      0.00      0.00         6

    accuracy                           0.53       146
   macro avg       0.32      0.36      0.33       146
weighted avg       0.51      0.53      0.51       146



/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [50]:
model = AdaBoostClassifier(n_estimators=200,algorithm='SAMME.R')
train_eval_model(model, X_train, y_train, X_test, y_test)

Train accuracy 0.607
Test accuracy 0.555
Train F1 Score 0.584
Test F1 Score 0.53

Cross val accuracy: 0.521
Cross val f1: 0.467

col_0  0  1   2   3
label              
0      1  4   0   0
1      0  8   7   2
2      0  1  16  38
3      0  0   7  56
4      0  0   0   6

              precision    recall  f1-score   support

           0       1.00      0.20      0.33         5
           1       0.62      0.47      0.53        17
           2       0.53      0.29      0.38        55
           3       0.55      0.89      0.68        63
           4       0.00      0.00      0.00         6

    accuracy                           0.55       146
   macro avg       0.54      0.37      0.38       146
weighted avg       0.54      0.55      0.51       146



/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Random Forest

In [51]:
model = RandomForestClassifier(random_state=0)
train_eval_model(model, X_train, y_train, X_test, y_test)

Train accuracy 1.0
Test accuracy 0.671
Train F1 Score 1.0
Test F1 Score 0.684

Cross val accuracy: 0.663
Cross val f1: 0.645

col_0  0   1   2   3
label               
0      5   0   0   0
1      2  10   5   0
2      0   2  35  18
3      0   0  15  48
4      0   0   0   6

              precision    recall  f1-score   support

           0       0.71      1.00      0.83         5
           1       0.83      0.59      0.69        17
           2       0.64      0.64      0.64        55
           3       0.67      0.76      0.71        63
           4       0.00      0.00      0.00         6

    accuracy                           0.67       146
   macro avg       0.57      0.60      0.57       146
weighted avg       0.65      0.67      0.66       146



/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
model = RandomForestClassifier(random_state=0)
train_eval_model(model, X_train, y_train, X_test, y_test,use_sample_weights=True)

Train accuracy 1.0
Test accuracy 0.685
Train F1 Score 1.0
Test F1 Score 0.69

Cross val accuracy: 0.663
Cross val f1: 0.645

col_0  0   1   2   3
label               
0      5   0   0   0
1      2  10   5   0
2      0   3  30  22
3      0   0   8  55
4      0   0   0   6

              precision    recall  f1-score   support

           0       0.71      1.00      0.83         5
           1       0.77      0.59      0.67        17
           2       0.70      0.55      0.61        55
           3       0.66      0.87      0.75        63
           4       0.00      0.00      0.00         6

    accuracy                           0.68       146
   macro avg       0.57      0.60      0.57       146
weighted avg       0.66      0.68      0.66       146



/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Gradient Boosting

In [53]:
model=GradientBoostingClassifier()
train_eval_model(model, X_train, y_train, X_test, y_test)

Train accuracy 1.0
Test accuracy 0.719
Train F1 Score 1.0
Test F1 Score 0.707

Cross val accuracy: 0.655
Cross val f1: 0.641

col_0  0  1   2   3  4
label                 
0      5  0   0   0  0
1      1  9   7   0  0
2      0  4  37  14  0
3      1  0   9  53  0
4      0  0   0   5  1

              precision    recall  f1-score   support

           0       0.71      1.00      0.83         5
           1       0.69      0.53      0.60        17
           2       0.70      0.67      0.69        55
           3       0.74      0.84      0.79        63
           4       1.00      0.17      0.29         6

    accuracy                           0.72       146
   macro avg       0.77      0.64      0.64       146
weighted avg       0.73      0.72      0.71       146



In [66]:
model=GradientBoostingClassifier()
train_eval_model(model, X_train, y_train, X_test, y_test)

Train accuracy 1.0
Test accuracy 0.658
Train F1 Score 1.0
Test F1 Score 0.656

Cross val accuracy: 0.919
Cross val f1: 0.917

col_0  0   1   2   3
label               
0      5   0   0   0
1      2  10   5   0
2      2   6  25  22
3      1   1   5  56
4      0   0   0   6

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         5
           1       0.59      0.59      0.59        17
           2       0.71      0.45      0.56        55
           3       0.67      0.89      0.76        63
           4       0.00      0.00      0.00         6

    accuracy                           0.66       146
   macro avg       0.49      0.59      0.51       146
weighted avg       0.64      0.66      0.63       146



/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
model=GradientBoostingClassifier()
train_eval_model(model, X_train, y_train, X_test, y_test,use_sample_weights=True)

Train accuracy 1.0
Test accuracy 0.658
Train F1 Score 1.0
Test F1 Score 0.653

Cross val accuracy: 0.656
Cross val f1: 0.635

col_0  0  1   2   3  4
label                 
0      5  0   0   0  0
1      3  9   5   0  0
2      0  5  33  16  1
3      1  0  15  47  0
4      0  0   0   4  2

              precision    recall  f1-score   support

           0       0.56      1.00      0.71         5
           1       0.64      0.53      0.58        17
           2       0.62      0.60      0.61        55
           3       0.70      0.75      0.72        63
           4       0.67      0.33      0.44         6

    accuracy                           0.66       146
   macro avg       0.64      0.64      0.61       146
weighted avg       0.66      0.66      0.65       146



## After Scaling Data with RobustScaler

### KNN

In [58]:
model = KNeighborsClassifier(n_neighbors=5)
train_eval_model(model, X_train_robustScale, y_train, X_test_robustScale, y_test)

Train accuracy 0.634
Test accuracy 0.425
Train F1 Score 0.618
Test F1 Score 0.429

Cross val accuracy: 0.435
Cross val f1: 0.421

col_0  0  1   2   3
label              
0      3  1   1   0
1      0  2  10   5
2      1  4  26  24
3      0  3  29  31
4      0  1   3   2

              precision    recall  f1-score   support

           0       0.75      0.60      0.67         5
           1       0.18      0.12      0.14        17
           2       0.38      0.47      0.42        55
           3       0.50      0.49      0.50        63
           4       0.00      0.00      0.00         6

    accuracy                           0.42       146
   macro avg       0.36      0.34      0.34       146
weighted avg       0.40      0.42      0.41       146



/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
model=KNeighborsClassifier()
param_grid = {'n_neighbors': np.arange(1, 200)}

gridSearch = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    verbose=10,
    cv=5
)
gridSearch.fit(X_train_robustScale,y_train)

Fitting 5 folds for each of 199 candidates, totalling 995 fits
[CV 1/5; 1/199] START n_neighbors=1.............................................
[CV 1/5; 1/199] END ..............n_neighbors=1;, score=0.342 total time=   0.0s
[CV 2/5; 1/199] START n_neighbors=1.............................................
[CV 2/5; 1/199] END ..............n_neighbors=1;, score=0.368 total time=   0.0s
[CV 3/5; 1/199] START n_neighbors=1.............................................
[CV 3/5; 1/199] END ..............n_neighbors=1;, score=0.371 total time=   0.0s
[CV 4/5; 1/199] START n_neighbors=1.............................................
[CV 4/5; 1/199] END ..............n_neighbors=1;, score=0.457 total time=   0.0s
[CV 5/5; 1/199] START n_neighbors=1.............................................
[CV 5/5; 1/199] END ..............n_neighbors=1;, score=0.405 total time=   0.0s
[CV 1/5; 2/199] START n_neighbors=2.............................................
[CV 1/5; 2/199] END ..............n_neighbors=

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92...
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
       183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195,
       196, 197, 198, 199])},
             scoring='accuracy', verbose=10)

In [60]:
gridSearch.best_params_

{'n_neighbors': 46}

In [61]:
gridSearch.best_score_

0.47422634836427935

In [62]:
model = KNeighborsClassifier(n_neighbors=46)
train_eval_model(model, X_train_robustScale, y_train, X_test_robustScale, y_test)

Train accuracy 0.515
Test accuracy 0.418
Train F1 Score 0.541
Test F1 Score 0.417

Cross val accuracy: 0.474
Cross val f1: 0.391

col_0   2   3
label        
0       4   1
1       9   8
2      10  45
3      12  51
4       0   6

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.00      0.00      0.00        17
           2       0.29      0.18      0.22        55
           3       0.46      0.81      0.59        63
           4       0.00      0.00      0.00         6

    accuracy                           0.42       146
   macro avg       0.15      0.20      0.16       146
weighted avg       0.31      0.42      0.34       146



/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Cap Outliers

In [ ]:
# XGBoost

model = xgb.XGBClassifier(
                       objective='multi:softmax',
                       random_state=50)      

train_eval_model(model, X_train_ext_cap, y_train, X_test_ext_cap, y_test)

In [ ]:
# Random Forest

model = RandomForestClassifier(random_state=0)
train_eval_model(model, X_train_ext_cap, y_train, X_test_ext_cap, y_test)

### Remove Corelated Features

In [ ]:
# XGBoost

model = xgb.XGBClassifier(
                       objective='multi:softmax',
                       random_state=50)      

train_eval_model(model, X_train_sel2, y_train, X_test_sel2, y_test)

In [ ]:
# Random Forest

model = RandomForestClassifier(random_state=0)
train_eval_model(model, X_train_sel2, y_train, X_test_sel2, y_test)

### Mutial Info Classification

In [ ]:
# XGBoost

model = xgb.XGBClassifier(
                       objective='multi:softmax',
                       random_state=50)      

train_eval_model(model, X_train_fsel_mic, y_train, X_test_fsel_mic, y_test)

In [ ]:
# Random Forest

model = RandomForestClassifier(random_state=0)
train_eval_model(model, X_train_fsel_mic, y_train, X_test_fsel_mic, y_test)

### Optimised Models

### XGBoost

In [91]:
model = xgb.XGBClassifier(
                       max_depth = 3,
                       n_estimators = 120,
                       learning_rate = 0.14,
                       gamma = 0.01,
                       reg_lambda = 0.5,
                       reg_alpha = 0.6,
                       min_child_weight=5,
                       random_state=100)      

train_eval_model(model, X_train_fsel_mic, y_train, X_test_fsel_mic, y_test)

Train accuracy 1.0
Test accuracy 0.74
Train F1 Score 1.0
Test F1 Score 0.73

Cross val accuracy: 0.636
Cross val f1: 0.618

col_0  0   1   2   3  4
label                  
0      5   0   0   0  0
1      0  13   4   0  0
2      0   3  37  15  0
3      0   1  10  52  0
4      0   0   0   5  1

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       0.76      0.76      0.76        17
           2       0.73      0.67      0.70        55
           3       0.72      0.83      0.77        63
           4       1.00      0.17      0.29         6

    accuracy                           0.74       146
   macro avg       0.84      0.69      0.70       146
weighted avg       0.75      0.74      0.73       146



### Gradient Boosting

In [95]:
model=GradientBoostingClassifier( n_estimators = 75,
                                  max_depth = 3,
                                  learning_rate = 0.16 )
    
train_eval_model(model, X_train_fsel_mic, y_train, X_test_fsel_mic, y_test)

Train accuracy 1.0
Test accuracy 0.705
Train F1 Score 1.0
Test F1 Score 0.722

Cross val accuracy: 0.662
Cross val f1: 0.665

col_0  0   1   2   3  4
label                  
0      5   0   0   0  0
1      1  11   5   0  0
2      0   2  40  13  0
3      1   1  13  47  1
4      0   0   0   6  0

              precision    recall  f1-score   support

           0       0.71      1.00      0.83         5
           1       0.79      0.65      0.71        17
           2       0.69      0.73      0.71        55
           3       0.71      0.75      0.73        63
           4       0.00      0.00      0.00         6

    accuracy                           0.71       146
   macro avg       0.58      0.62      0.60       146
weighted avg       0.68      0.71      0.69       146



### Random Forest

In [128]:
model=RandomForestClassifier(n_estimators = 100,
                             max_depth = 6,
                             random_state=0)
train_eval_model(model, X_train_fsel_mic, y_train, X_test_fsel_mic, y_test)

Train accuracy 0.897
Test accuracy 0.699
Train F1 Score 0.882
Test F1 Score 0.708

Cross val accuracy: 0.663
Cross val f1: 0.643

col_0  0   1   2   3
label               
0      5   0   0   0
1      1  10   6   0
2      0   2  33  20
3      0   0   9  54
4      0   0   0   6

              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       0.83      0.59      0.69        17
           2       0.69      0.60      0.64        55
           3       0.68      0.86      0.76        63
           4       0.00      0.00      0.00         6

    accuracy                           0.70       146
   macro avg       0.61      0.61      0.60       146
weighted avg       0.68      0.70      0.68       146



/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
